# Data Collection

## PVGIS
- https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis_en
- Application-specific aggregator of irradiance data
- TMY
- Time-series irradiance
- PV Simulation
- Horizon Profile

Irradiance data is from ERA5, NSRDB, SARAH3 databases, which are reeanalysis products

Access through website or web API or using pvlib

## NSRDB

PSM
- Satellites GOES, IMS, MODIS
- MERRA-2 reeanalysis
- Cloud and aerosol properties are calculated and resampled
- Fed to a radiative transfer model, Fast all-sky Radiation Model for Solar Applications
- 

## Copernicus ADS CAMS

## Other
- Solar Anywhere
- Meteomatics
- Solcast
- Solargis
- Baseline Surface Radiation Network
- NOAA Applied Climate Information System
- University of Oregon Solar Radiation Monitoring Laboratory

In [ ]:
import pvlib
import json
import os
from pprint import pprint
from datetime import datetime
from pytz import timezone
import statistics

import pandas as pd


In [ ]:
def get_pvgis_tmy_from_pvgis(lat, lon, timezone, startyear, endyear):

    key = 'pvgis_tmy'
    if os.path.exists(key) is False:
        os.mkdir(key)

    filename = f"{key}/pvgis_tmy_{lat}_{lon}_{startyear}_{endyear}.csv"
    metadata_filename = f"{key}/pvgis_tmy_{lat}_{lon}_{startyear}_{endyear}.json"

    if os.path.exists(filename) is False:
        df, metadata = pvlib.iotools.get_pvgis_tmy(
            lat,
            lon,
            outputformat='json',
            usehorizon=True,
            startyear=startyear,
            endyear=endyear,
            map_variables=True,
            timeout=30
        )
    
        df.to_csv(filename)
        with open(metadata_filename, mode='w') as f:
            json.dump(metadata, f)

    df = pd.read_csv(filename)
    with open(metadata_filename, mode='r') as f:
        metadata = json.load(f)

    df["Year"] = 1990
    df["Month"] = pd.DatetimeIndex(df["time(UTC)"]).month
    df["Day"] = pd.DatetimeIndex(df["time(UTC)"]).day
    df["Hour"] = pd.DatetimeIndex(df["time(UTC)"]).hour
    df["Minute"] = pd.DatetimeIndex(df["time(UTC)"]).minute
    df["sitetime"] = df.apply(lambda x: datetime(int(x["Year"]), int(x["Month"]), int(x["Day"]), int(x["Hour"]), int(x["Minute"])), axis=1)

    df.index = pd.DatetimeIndex(df["sitetime"], tz="UTC").tz_convert(timezone)
    df.drop(columns=['time(UTC)'], inplace=True)

    df = df[['temp_air', 'dhi', 'dni', 'ghi', 'pressure', 'wind_direction', 'wind_speed']]
    
    return df, metadata

In [ ]:
def get_tmy_from_nsrdb(lat, lon, timezone, api_key, full_name, email, affiliation, data_source='himawari'):

    if data_source == "himawari":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/himawari-tmy-download.csv"
    elif data_source == "goes":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/nsrdb-GOES-tmy-v4-0-0-download.csv"
    elif data_source == "meteosat":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/nsrdb-msg-v1-0-0-tmy-download.csv"
    else:
        raise ValueError(f"Invalid data source {data_source}")

    key = f'{data_source}_tmy'
    if os.path.exists(key) is False:
        os.mkdir(key)

    filename = f"{key}/{data_source}_tmy_{lat}_{lon}.csv"
    metadata_filename = f"{key}/{data_source}_tmy_{lat}_{lon}.json"

    if os.path.exists(filename) is False:
        df, metadata = pvlib.iotools.get_nsrdb_psm4_tmy(
            lat,
            lon,
            api_key,
            email,
            year='tmy',
            time_step=60,
            leap_day=False,
            full_name=full_name,
            affiliation=affiliation,
            utc=True,
            map_variables=True,
            url=url,
            timeout=30
        )
    
        df.to_csv(filename)
        with open(metadata_filename, mode='w') as f:
            json.dump(metadata, f)

    df = pd.read_csv(filename, index_col=0)
    with open(metadata_filename, mode='r') as f:
        metadata = json.load(f)

    df["Year"] = 1990
    df["sitetime"] = df.apply(lambda x: datetime(int(x["Year"]), int(x["Month"]), int(x["Day"]), int(x["Hour"]), int(x["Minute"])), axis=1)
    df.index = pd.DatetimeIndex(df["sitetime"], tz="UTC").tz_convert(timezone)


    df = df[['temp_air', 'temp_dew', 'dhi', 'dni', 'ghi', 'albedo', 'pressure', 'wind_direction', 'wind_speed'
    ]]
    
    return df, metadata

In [ ]:
api_key = '9nbF8PWcvcjXeQALyMXmOfmefnOTb942xxHGvR0M'
full_name = "David Pratama Widjaja"
affiliation = ""
email = "widjaja.david@ymail.com"
reason = 'Research into Indonesian and ASEAN energy transition'
join_mailing_list = 'true'

jakarta_lat = -6.175389
jakarta_lon = 106.827139
jakarta_data_source = 'himawari'
jakarta_tz = timezone('Asia/Jakarta')

amsterdam_lat = 52.372778
amsterdam_lon = 4.893611
amsterdam_data_source = 'meteosat'
amsterdam_tz = timezone('Europe/Amsterdam')

surabaya_lat = -7.245833
surabaya_lon = 112.737778
surabaya_data_source = 'himawari'
surabaya_tz = timezone('Asia/Makassar')

vancouver_lat = 49.260833
vancouver_lon = -123.113889
vancouver_data_source = 'goes'
vancouver_tz = timezone('America/Vancouver')

toronto_lat = 43.741667
toronto_lon = -79.373333
toronto_data_source = 'goes'
toronto_tz = timezone('America/Toronto')

In [ ]:
#weather_df, weather_metadata = get_tmy_from_nsrdb(jakarta_lat, jakarta_lon, jakarta_tz, api_key, full_name, email, affiliation, data_source=jakarta_data_source)
#df = get_tmy_from_nsrdb(jakarta_lat, jakarta_lon, api_key, full_name, email, affiliation, data_source='himawari')
#df = get_tmy_from_nsrdb(amsterdam_lat, amsterdam_lon, api_key, full_name, email, affiliation, data_source='meteosat')

weather_df, pvgis_metadata = get_pvgis_tmy_from_pvgis(jakarta_lat, jakarta_lon, jakarta_tz, 2005, 2023)

In [ ]:
# now, get the monthly ghi

In [ ]:
# get possibilities for module manufacturer and technology

modules_df = pvlib.pvsystem.retrieve_sam(name='CECMod').T.reset_index(names="Model")

In [ ]:
inverters_df = pvlib.pvsystem.retrieve_sam(name='CECInverter').T.reset_index(names='Model')

## What modules to use in Indonesia?

- Must be Trina Solar, because of domestic content obligation
- Must be bifacial, because of limited space and because of open-rack mounting
- Must be a Trina Solar product manufactured in Indonesia, so
-     Vertex N 700W TSM-NEG21C.20
-     Vertex N 610W+Bifacial NEG19RC.20
-     Vertex S+ 450W TSM – NEG9R.28

Of these, only the **Vertex N 610W+Bifacial NEG19RC.20** is bifacial.

## How to get inverter params?

We can either use SAPM or CEC parameters

### SAPM(Sandia Array Performance Model)

### CEC

Name,
Technology,
Bifacial,
STC,
PTC,
A_c,
Length,
Width,

N_s,
I_sc_ref,
V_oc_ref,
I_mp_ref,
V_mp_ref,
alpha_sc,
beta_oc,
T_NOCT,
a_ref,
I_L_ref,
I_o_ref,
R_s,
R_sh_ref,
Adjust,
gamma_r,
BIPV,
Version,
Date



In [ ]:
# I_L_ref (float) – The light-generated current (or photocurrent) at reference conditions [A]
# I_o_ref (float) – The dark or diode reverse saturation current at reference conditions [A]
# R_s (float) – The series resistance at reference conditions, in ohms.
# R_sh_ref (float) – The shunt resistance at reference conditions, in ohms.
# a_ref (float) – The product of the usual diode ideality factor n (unitless), number of cells in series Ns, and cell thermal voltage at reference conditions [V]
# Adjust (float) – The adjustment to the temperature coefficient for short circuit current, in percent.

celltype='monoSi'
v_mp = 39.43
i_mp = 15.98
v_oc = 47.69
i_sc = 16.82
alpha_sc = (0.04/100) * 16.82
beta_voc = -(0.24/100) * 47.69
gamma_pmp = -0.29
cells_in_series=132
temp_ref=25
length = 2.382
width = 1.134
module_area = length * width
module_count = 10

I_L_ref, I_o_ref, R_s, R_sh_ref, a_ref, Adjust = pvlib.ivtools.sdm.fit_cec_sam(
    celltype=celltype,
    v_mp=v_mp,
    i_mp=i_mp,
    v_oc=v_oc,
    i_sc=i_sc,
    alpha_sc=alpha_sc,
    beta_voc=beta_voc,
    gamma_pmp=gamma_pmp,
    cells_in_series=cells_in_series,
    temp_ref=temp_ref
)

cec_module_parameters = {
    "Name": "Vertex",
    "Technology": "Mono-c-Si",
    "Bifacial": 1,
    "STC": 605,
    "PTC": 462,
    "A_c": 2.382 * 1.134,
    "Length": 1.134,
    "Width": 2.382,
    "N_s": cells_in_series,
    "I_sc_ref": i_sc,
    "V_oc_ref": v_oc,
    "I_mp_ref": i_mp,
    "V_mp_ref": v_mp,
    "alpha_sc": alpha_sc,
    "beta_oc": beta_voc,
    "T_NOCT": 20,
    "a_ref": a_ref,
    "I_L_ref": I_L_ref,
    "I_o_ref": I_o_ref,
    "R_s": R_s,
    "R_sh_ref": R_sh_ref,
    "Adjust": Adjust,
    "gamma_r": gamma_pmp,
    "BIPV": "Y",
    "Version": 1,
    "Date": datetime(2024, 5, 1)
}

inverter_parameters = {
    'Model': 'ABB__UNO_DM_5_0_TL_PLUS_US_SZ_RA__208V_',
    'Vac': 208,
    'Pso': 27.736912,
    'Paco': 4600.0,
    'Pdco': 4776.102051,
    'Vdco': 340.0,
    'C0': -3.378698e-06,
    'C1': -6.160622e-06,
    'C2': 0.002079,
    'C3': 0.000439,
    'Pnt': 1.0,
    'Vdcmax': 480.0,
    'Idcmax': 14.047359,
    'Mppt_low': 240.0,
    'Mppt_high': 480.0,
    'CEC_Date': '2/15/2019',
    'CEC_Type': 'Grid Support'
}

In [ ]:
# create an array

location = pvlib.location.Location(jakarta_lat, jakarta_lon, tz=jakarta_tz, altitude=0, name='jakarta location')


mount = pvlib.pvsystem.FixedMount(
    surface_tilt=20.0,
    surface_azimuth=0.0,
    racking_model='open_rack',
    module_height=2.0
)

array = pvlib.pvsystem.Array(
    mount,
    albedo=None,
    surface_type='concrete',
    module=None,
    module_type='glass_glass',
    module_parameters=cec_module_parameters,
    temperature_model_parameters=None,
    modules_per_string=10,
    strings=1,
    array_losses_parameters=None,
    name='Vertex N 610W+Bifacial NEG19RC.20'
)

pvsystem = pvlib.pvsystem.PVSystem(
    arrays=array,
    surface_tilt=20,
    surface_azimuth=0,
    albedo=None,
    surface_type='concrete',
    module=None,
    module_type='glass_glass',
    module_parameters=cec_module_parameters,
    temperature_model_parameters=None,
    modules_per_string=10,
    strings_per_inverter=1,
    inverter="ABB__UNO_DM_5_0_TL_PLUS_US_SB_RA__208V_",
    inverter_parameters=inverter_parameters,
    racking_model='open_rack',
    losses_parameters=None,
    name='Jakarta PVSystem'
)

modelchain = pvlib.modelchain.ModelChain(
    pvsystem,
    location,
    clearsky_model='ineichen',
    transposition_model='haydavies',
    solar_position_method='nrel_numpy',
    airmass_model='kastenyoung1989',
    dc_model='cec',
    ac_model='sandia',
    aoi_model='physical',
    spectral_model='no_loss',
    temperature_model='sapm',
    dc_ohmic_model='no_loss',
    losses_model='no_loss',
    name="Jakarta Modelchain"
)


In [ ]:
modelchain.run_model(weather_df)

In [ ]:
modelchain.results.total_irrad.plot()

In [ ]:
ac = modelchain.results.ac

In [ ]:
#ac[datetime(1990, 1, 1, tzinfo=jakarta_tz):datetime(1990, 2, 1, tzinfo=jakarta_tz)].resample('1d').plot
(ac.resample('1MS').sum() / 1000).plot.bar()

In [ ]:

electric_car_battery_capacity_kwh = {
    "BYD Seal Premium": 82.56,
    "BYD Seal Performance": 82.56,
    "BYD Atto 3 Advanced": 49.92,
    "BYD Atto 3 Superior": 60.48,
    "BYD Dolphin Dynamic": 44.9,
    "BYD Dolphin Premium": 60.48,
    "BYD M6 Standard": 55.4,
    "BYD M6 Superior": 71.8,
    "BYD M6 Superior Captain": 71.8,
    "BYD Sealion 7 Premium": 82.56,
    "BYD Sealion 7 Performance": 82.56,
    "Wuling New Air EV Lite 200KM": 17.3,
    "Wuling New Air EV Lite 300KM": 26.7,
    "Wuling New Air EV Pro 300KM": 26.7,
    "Wuling BinguoEV 410KM Premium Range": 31.9,
    "Wuling BinguoEV 333KM Long Range": 31.9,
    "Wuling BinguoEV 333KM Long Range": 31.9,
    "Wuling New Cloud EV Pro": 50.6,
    "Wuling New Cloud EV Lite": 50.6,
    "BMW i5 eDrive40 Sedan": 81.2,
    "BMW i5 eDrive40 Limousine": 81.2,
    "BMW i5 M60 xDrive": 81.2,
}


In [ ]:
(263.32 * 1000 * 1000 * 1000 * 27.01 / 10206.19) / 1000000

In [ ]:
total_energy_produced_kwh = ac.sum() / 1000
total_area_m2 = module_count * module_area
total_dc_capacity = v_mp * i_mp * module_count / 1000
total_ac_capacity = inverter_parameters["Paco"] / 1000

electricity_tariff_rp_kwh = 3500

average_battery_charge = statistics.mean(list(electric_car_battery_capacity_kwh.values())) * 0.70
number_of_battery_charges = total_energy_produced_kwh / average_battery_charge
total_electricity_sold = electricity_tariff_rp_kwh * total_energy_produced_kwh

number_of_ev_stations = 2
number_of_chargers_per_station_per_day = 3.5
total_energy_fast_charging_kwh = number_of_ev_stations * number_of_chargers_per_station_per_day * average_battery_charge * 365
total_electricity_sold_gross = total_energy_fast_charging_kwh * (electricity_tariff_rp_kwh - 1000)

print("Total DC Capacity Installed(kWp): ", round(total_dc_capacity))
print("Total AC Capacity Installed(kWac): ", total_ac_capacity)
print("Total Energy Produced(kWh): ", round(total_energy_produced_kwh, 2))
print("Total Area(m): ", round(total_area_m2, 2))
print("Average Battery Charge(kWh): ", round(average_battery_charge))
print("Number of Battery Charges: ", round(number_of_battery_charges))
print("Electricity Sold(Rp): ", round(total_electricity_sold / 1000)* 1000)

print("\nNumber of Charges: ", round(number_of_ev_stations * number_of_chargers_per_station_per_day * 365))
print("Total Energy Produced(kWh): ", round(total_energy_fast_charging_kwh, 2))
print("Electricity Sold(Rp): ", round(total_electricity_sold_gross / 1000)* 1000)

print("\nRatio: ", round(total_energy_produced_kwh / total_energy_fast_charging_kwh, 2))

In [ ]:
ac.plot()